# Lesson 1: Simple ReAct Travel Agent from Scratch

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [4]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [5]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [6]:
prompt = """
You are a helpful travel assistant that runs in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your reasoning about the travel question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_flight_info:
e.g. get_flight_info: New York to London
Returns flight duration and typical cost between two cities

get_weather:
e.g. get_weather: Paris
Returns current weather information for a destination

calculate:
e.g. calculate: 450 * 2 + 100
Runs a calculation and returns the number - useful for travel budgets, costs, etc.

Example session:

Question: What's the weather like in Paris and how much would a flight from New York cost?
Thought: I need to get weather information for Paris and flight information from New York to Paris
Action: get_weather: Paris
PAUSE

You will be called again with this:

Observation: Currently 18°C, partly cloudy. Good weather for sightseeing!

Then continue:
Thought: Now I need to get flight information from New York to Paris
Action: get_flight_info: New York to Paris
PAUSE

You will be called again with this:

Observation: Flight duration: 8 hours, Typical cost: $700-900

You then output:

Answer: The weather in Paris is currently 18°C and partly cloudy - great for sightseeing! A flight from New York to Paris typically takes 8 hours and costs between $700-900.
""".strip()

In [7]:
def calculate(what):
    return eval(what)

def get_flight_info(route):
    """Get flight information between two cities"""
    route = route.strip()
    
    # Simple mock data for common routes
    flight_data = {
        "New York to London": "Flight duration: 7 hours, Typical cost: $600-800",
        "London to Paris": "Flight duration: 1.5 hours, Typical cost: $150-250", 
        "New York to Paris": "Flight duration: 8 hours, Typical cost: $700-900",
        "Tokyo to London": "Flight duration: 12 hours, Typical cost: $800-1200",
        "Los Angeles to Tokyo": "Flight duration: 11 hours, Typical cost: $600-1000",
        "London to New York": "Flight duration: 8 hours, Typical cost: $600-800",
        "Paris to London": "Flight duration: 1.5 hours, Typical cost: $150-250",
        "Paris to New York": "Flight duration: 8.5 hours, Typical cost: $700-900"
    }
    
    return flight_data.get(route, f"Flight information not available for {route}. Typical international flights cost $500-1000 and take 6-12 hours.")

def get_weather(city):
    """Get weather information for a destination"""
    city = city.strip()
    
    # Simple mock weather data
    weather_data = {
        "Paris": "Currently 18°C, partly cloudy. Good weather for sightseeing!",
        "London": "Currently 15°C, light rain. Pack an umbrella!",
        "New York": "Currently 22°C, sunny. Perfect weather for walking around the city!",
        "Tokyo": "Currently 25°C, humid. Light clothing recommended.",
        "Los Angeles": "Currently 28°C, sunny. Great beach weather!",
        "Rome": "Currently 24°C, sunny. Perfect for exploring ancient sites!",
        "Barcelona": "Currently 26°C, clear skies. Ideal for beach and city tours!"
    }
    
    return weather_data.get(city, f"Weather information not available for {city}. Check local weather services for current conditions.")

known_actions = {
    "calculate": calculate,
    "get_flight_info": get_flight_info,
    "get_weather": get_weather
}

In [8]:
abot = Agent(prompt)

In [9]:
result = abot("What's the weather like in Paris?")
print(result)

Thought: I need to get the current weather information for Paris.
Action: get_weather: Paris
PAUSE


In [10]:
result = get_weather("Paris")

In [11]:
result

'Currently 18°C, partly cloudy. Good weather for sightseeing!'

In [12]:
next_prompt = "Observation: {}".format(result)

In [13]:
abot(next_prompt)

'Answer: The weather in Paris is currently 18°C and partly cloudy, which is good weather for sightseeing!'

In [14]:
abot.messages

[{'role': 'system',
  'content': "You are a helpful travel assistant that runs in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\n\nUse Thought to describe your reasoning about the travel question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_flight_info:\ne.g. get_flight_info: New York to London\nReturns flight duration and typical cost between two cities\n\nget_weather:\ne.g. get_weather: Paris\nReturns current weather information for a destination\n\ncalculate:\ne.g. calculate: 450 * 2 + 100\nRuns a calculation and returns the number - useful for travel budgets, costs, etc.\n\nExample session:\n\nQuestion: What's the weather like in Paris and how much would a flight from New York cost?\nThought: I need to get weather information for Paris and flight information from New York to Paris\nAction: 

In [15]:
abot = Agent(prompt)

In [16]:
question = """I'm planning a trip from New York to London. What's the flight cost and duration, \
and what's the weather like in London? If the flight costs $700, what would be the total cost for 2 people?"""
abot(question)

'Thought: I need to get flight information from New York to London and also check the current weather in London. After that, I will calculate the total cost for 2 people if the flight costs $700 each.\nAction: get_flight_info: New York to London\nPAUSE'

In [17]:
next_prompt = "Observation: {}".format(get_flight_info("New York to London"))
print(next_prompt)

Observation: Flight duration: 7 hours, Typical cost: $600-800


In [18]:
abot(next_prompt)

'Thought: Now I need to get the current weather information for London.\nAction: get_weather: London\nPAUSE'

In [19]:
next_prompt = "Observation: {}".format(get_weather("London"))
print(next_prompt)

Observation: Currently 15°C, light rain. Pack an umbrella!


In [20]:
abot(next_prompt)

'Thought: I have the flight information and the weather details. Now, I need to calculate the total cost for 2 people if each flight costs $700.\nAction: calculate: 700 * 2\nPAUSE'

In [21]:
next_prompt = "Observation: {}".format(eval("700 * 2"))
print(next_prompt)

Observation: 1400


In [22]:
abot(next_prompt)

"Answer: The flight from New York to London typically takes 7 hours and costs between $600-800. The current weather in London is 15°C with light rain, so it's advisable to pack an umbrella. If the flight costs $700 per person, the total cost for 2 people would be $1,400."

### Add loop 

In [23]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [24]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [25]:
question = """What's the weather like in Paris?"""
query(question)

Thought: I need to get the current weather information for Paris.
Action: get_weather: Paris
PAUSE
 -- running get_weather Paris
Observation: Currently 18°C, partly cloudy. Good weather for sightseeing!
Answer: The weather in Paris is currently 18°C and partly cloudy, which is good weather for sightseeing!


In [26]:
question = """I want to travel from New York to London. Can you tell me the flight duration and cost, \
plus what's the weather like in London? If flights cost $750 each, what's the total for 2 travelers?"""
query(question)

Thought: I need to get flight information from New York to London and the current weather in London. Then, I will calculate the total cost for 2 travelers if each flight costs $750.
Action: get_flight_info: New York to London
PAUSE
 -- running get_flight_info New York to London
Observation: Flight duration: 7 hours, Typical cost: $600-800
Thought: Now I need to get the current weather information for London.
Action: get_weather: London
PAUSE
 -- running get_weather London
Observation: Currently 15°C, light rain. Pack an umbrella!
Thought: I have the flight duration, cost, and weather information. Now, I need to calculate the total cost for 2 travelers if each flight costs $750.
Action: calculate: 750 * 2
PAUSE
 -- running calculate 750 * 2
Observation: 1500
Answer: The flight from New York to London typically takes 7 hours and costs between $600-800. The current weather in London is 15°C with light rain, so it's advisable to pack an umbrella. If each flight costs $750, the total cost f